In [253]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
import sklearn.pipeline as pipe
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.svm import SVC
import nltk

In [262]:
#using pandas to import the original training_text and traning_variant
sms = pd.read_table('training__frequent2_text.txt', encoding= 'UTF8', sep="\\|\\|",header=None, engine='python', names=['cli_data'])
sms2= pd.read_table('training_variants.txt', encoding= 'UTF8', header=None,delimiter=',', engine='python', names=['gene','mutation','classification'])

In [263]:
sms.shape

(3321, 1)

In [264]:
#a glimpse of the type, doesn't matter
type(sms)

pandas.core.frame.DataFrame

In [265]:
sms2.shape

(3321, 3)

In [266]:
#the first 10 lines of the text file
sms.head(10)

,cli_data
0,Cyclin kinases CDKs regulate variety fundament...
1,Abstract Background Non lung NSCLC heterogeneo...
2,Abstract Background Non lung NSCLC heterogeneo...
3,Recent uniparental disomy aUPD pathogenetic ho...
4,Oncogenic monomeric Casitas B lineage lymphoma...
5,Oncogenic monomeric Casitas B lineage lymphoma...
6,Oncogenic monomeric Casitas B lineage lymphoma...
7,CBL regulator tyrosine kinases RTK In CBL acut...
8,Abstract Juvenile myelomonocytic leukemia JMML...
9,Abstract Juvenile myelomonocytic leukemia JMML...


In [267]:
#first 10 lines of the variants file
sms2.head(10)

,gene,mutation,classification
0,FAM58A,Truncating Mutations,1
1,CBL,W802*,2
2,CBL,Q249E,2
3,CBL,N454D,3
4,CBL,L399V,4
5,CBL,V391I,4
6,CBL,V430M,5
7,CBL,Deletion,1
8,CBL,Y371H,4
9,CBL,C384R,4


In [268]:
#the classification is very imbalanced use this dictinaty as the "class-weight" parameter when do model fitting
weighted=sms2.classification.value_counts()
weight_dict=dict()
for key in weighted.keys():
    weight_dict[key]=weighted[key]/3311
print(weight_dict)

{7: 0.2878284506191483, 4: 0.20718816067653276, 1: 0.17154938085170643, 2: 0.13651464814255512, 6: 0.083056478405315617, 5: 0.073089700996677748, 3: 0.026880096647538508, 9: 0.01117487163998792, 8: 0.0057384475989127152}


In [269]:
#define the X and y to be feed to the model
X = sms.cli_data
y = sms2.classification

In [271]:
#split the training data into "train" and "test" when run the model
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2490,)
(831,)
(2490,)
(831,)


In [ ]:
#using kfold to split the data
import re
kf=KFold(n_splits=4)
array_nb=[]
array_lr=[]
array_svc=[]
score_nb=0
score_lr=0
score_svc=0
i=0
for train_indices,test_indices in kf.split(X,y):
    X_train=X[train_indices]
    y_train=y[train_indices] 
    X_test=X[test_indices] 
    y_test=y[test_indices] 
    
    vect = CountVectorizer()
    X_train_dtm = vect.fit_transform(X_train)
    X_test_dtm = vect.transform(X_test)
    
    nb = MultinomialNB()
    nb.fit(X_train_dtm, y_train)
    y_pred_class_nb = nb.predict(X_test_dtm)
    score_nb+=metrics.accuracy_score(y_test, y_pred_class_nb)
    array_nb.append(metrics.accuracy_score(y_test, y_pred_class_nb))
    
    logreg = LogisticRegression()
    logreg.fit(X_train_dtm, y_train)
    y_pred_class_lr = logreg.predict(X_test_dtm)
    score_lr+=metrics.accuracy_score(y_test, y_pred_class_lr)
    array_lr.append(metrics.accuracy_score(y_test, y_pred_class_lr))
    
    svc = SVC(class_weight=weight_dict)
    svc.fit(X_train_dtm, y_train)
    y_pred_class_svc = svc.predict(X_test_dtm)
    score_svc+=metrics.accuracy_score(y_test, y_pred_class_svc)
    array_svc.append(metrics.accuracy_score(y_test, y_pred_class_svc))
    #knn.fit(X_train_dtm, y_train)
    #y_pred_class_knn= knn.predict(X_test_dtm)
    #score_knn+=metrics.accuracy_score(y_test, y_pred_class_knn)
    #array_knn.append(metrics.accuracy_score(y_test, y_pred_class_knn))
    i=i+1



    

In [ ]:

print(score_nb)
print(score_lr)
print(score_svc)
print(i)
print(array_nb)
print(array_lr)
print(array_svc)

In [ ]:
accuracy_nb=score_nb/i
accuracy_lr=score_lr/i
accuracy_knn=score_knn/i
print(accuracy_nb)
print(accuracy_lr)


In [ ]:
#clf = pipe.make_pipeline(TfidfVectorizer(), MultinomialNB())
#clf.fit_tansform(X,y)
#score=cross_val_score(clf, X, y, cv=10, scoring='accuracy')
#print(score)

In [272]:
#create the vectorized instance
vect = CountVectorizer()

In [273]:
#fit--learn the train vocabulary, as "features";transform is to create a "document term matrix"
X_train_dtm = vect.fit_transform(X_train)
X_train_dtm

<2490x137259 sparse matrix of type '<class 'numpy.int64'>'
	with 3328599 stored elements in Compressed Sparse Row format>

In [274]:
#do the same thing to test file 
X_test_dtm = vect.transform(X_test)
X_test_dtm 

<831x137259 sparse matrix of type '<class 'numpy.int64'>'
	with 1027661 stored elements in Compressed Sparse Row format>

In [275]:
#use naive bayes model to test
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [252]:
#accuracy using cross-validation
y_pred_class_nb = nb.predict(X_test_dtm)
metrics.accuracy_score(y_test, y_pred_class_nb)

0.57761732851985559

In [276]:
#the first 10 lines of  prediction
print(y_pred_class_nb[:10])

[1 7 1 7 1 1 1 2 4 1]


In [277]:
metrics.confusion_matrix(y_test, y_pred_class_nb)
#class 2&7 1&4 are easy to get confused

array([[ 87,   7,   0,  16,  18,   5,   9,   0,   0],
       [  5,  50,   0,   5,   1,   1,  55,   0,   0],
       [  4,   1,  11,   2,   3,   1,   5,   0,   0],
       [ 56,   3,   1,  86,  16,   3,   4,   0,   1],
       [  9,   2,   0,   3,  24,   4,   7,   0,   1],
       [ 10,   5,   1,   0,   6,  35,   4,   0,   0],
       [  5,  55,   6,   1,   4,   1, 178,   0,   0],
       [  1,   1,   0,   0,   0,   0,   1,   1,   1],
       [  0,   0,   0,   0,   0,   0,   1,   0,   8]])

In [283]:
#first 10 lines of classification prediction probality 
y_pred_prob = nb.predict_proba(X_test_dtm)[:10, 0]
y_pred_prob

array([  1.00000000e+000,   0.00000000e+000,   1.00000000e+000,
         9.83734304e-121,   9.99993072e-001,   1.00000000e+000,
         1.00000000e+000,   0.00000000e+000,   3.56773206e-222,
         1.00000000e+000])

In [285]:
#use logistic regression model to test
logreg = LogisticRegression(class_weight=weight_dict)
logreg.fit(X_train_dtm, y_train)

LogisticRegression(C=1.0,
          class_weight={7: 0.2878284506191483, 4: 0.20718816067653276, 1: 0.17154938085170643, 2: 0.13651464814255512, 6: 0.083056478405315617, 5: 0.073089700996677748, 3: 0.026880096647538508, 9: 0.01117487163998792, 8: 0.0057384475989127152},
          dual=False, fit_intercept=True, intercept_scaling=1,
          max_iter=100, multi_class='ovr', n_jobs=1, penalty='l2',
          random_state=None, solver='liblinear', tol=0.0001, verbose=0,
          warm_start=False)

In [286]:
y_pred_class = logreg.predict(X_test_dtm)

In [289]:
#predicted probiblity of each class for the first 10 lines
y_pred_prob_lr = logreg.predict_proba(X_test_dtm)[:10]
print(y_pred_prob_lr)

[[  9.99997865e-01   2.74858220e-12   1.83499180e-17   1.00522053e-09
    7.68455762e-09   2.06479588e-06   6.16043769e-08   2.60412160e-18
    7.44905648e-21]
 [  3.45306518e-11   2.86233827e-07   1.59732100e-04   5.16886091e-10
    3.91616053e-06   8.30794200e-11   9.99836065e-01   2.34681316e-10
    2.05826693e-13]
 [  9.99692847e-01   3.56467680e-05   4.08034927e-07   4.14786943e-05
    1.33393561e-05   1.54042363e-04   6.19435445e-05   2.92765817e-07
    1.64700455e-09]
 [  2.23456962e-04   3.01682406e-02   2.49163238e-08   3.22869624e-05
    1.33917263e-01   6.82122681e-02   7.67446457e-01   1.20179825e-10
    2.46701632e-09]
 [  2.13115726e-01   3.58635850e-14   1.34874489e-08   7.86865075e-01
    1.91797617e-05   1.13336212e-09   5.15393873e-09   3.94665211e-15
    5.39100722e-15]
 [  4.72813102e-01   6.06787668e-02   2.19315071e-03   4.37203563e-01
    2.32775107e-02   2.33600380e-03   1.06182395e-03   2.42572552e-04
    1.93506972e-04]
 [  1.02983744e-03   4.45721423e-07   1.

In [287]:
#the accuracy using log_regression model
metrics.accuracy_score(y_test, y_pred_class)

0.6305655836341757

In [293]:
#the inaccuray mainly happened between class 1&4, 2&7
metrics.confusion_matrix(y_test, y_pred_class)

array([[ 82,   4,   0,  34,   7,   7,   8,   0,   0],
       [  5,  55,   0,   4,   1,   4,  48,   0,   0],
       [  1,   0,   1,  10,   1,   0,  14,   0,   0],
       [ 25,   1,   0, 124,  12,   0,   8,   0,   0],
       [ 15,   0,   0,   9,  10,   5,  11,   0,   0],
       [  9,   4,   1,   0,   3,  41,   3,   0,   0],
       [  4,  32,   0,   5,   4,   3, 202,   0,   0],
       [  0,   0,   0,   0,   0,   0,   2,   3,   0],
       [  0,   0,   0,   2,   0,   0,   1,   0,   6]])

In [290]:
svc=SVC(class_weight=weight_dict)
svc.fit(X_train_dtm,y_train)

SVC(C=1.0, cache_size=200,
  class_weight={7: 0.2878284506191483, 4: 0.20718816067653276, 1: 0.17154938085170643, 2: 0.13651464814255512, 6: 0.083056478405315617, 5: 0.073089700996677748, 3: 0.026880096647538508, 9: 0.01117487163998792, 8: 0.0057384475989127152},
  coef0=0.0, decision_function_shape='ovr', degree=3, gamma='auto',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [291]:
y_pred_class_svc=svc.predict(X_test_dtm)

In [292]:
#the accuary of svc model is the lowest
metrics.accuracy_score(y_test, y_pred_class_svc)

0.41034897713598073

In [294]:
#use KNN model to test

In [295]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train_dtm, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')

In [296]:
y_pred_class_knn= knn.predict(X_test_dtm)

In [297]:
#get the accuracy using knn model
metrics.accuracy_score(y_test, y_pred_class_knn)

0.60409145607701564